In [4]:
#Need to add parent directoy to sys.path to find 'metadataDB'
import sys
sys.path.append('../')

%matplotlib inline
# import matplotlib.pyplot as plt 
import time
# import numpy as np
# import scipy as sp
import re
from collections import Counter
import random

# Natural language processing toolkit
# To use this, run nltk.download() and download 'stopwords'
# from nltk.corpus import stopwords
# s=stopwords.words('english') + ['']

# Machine learning
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.feature_extraction.text import TfidfTransformer
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.linear_model import LogisticRegression
# from sklearn.svm import LinearSVC, SVC
# from sklearn.pipeline import Pipeline
# from sklearn import metrics

# SQL
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from metadataDB.declareDatabase import *
from sqlalchemy import or_, and_

engine = create_engine("sqlite:///../arXiv_metadata.db", echo=False)
Base.metadata.bind = engine
DBsession = sessionmaker(bind=engine)
session = DBsession()

In [5]:
query = session.query(Article_Category)\
                    .join(Category)\
                    .join(Article)\
                    .filter(Category.name.like('%quant-gas'))
resultPRL = query.all()
abstract_all = [x.article.abstract for x in query.all()]
session.close_all()

In [6]:
abstract_train = abstract_all
print len(abstract_train)

9107


CountVectorizer might do a cleaner job.

In [7]:
class AbstractWriter:
    '''This class will take a specific n-gram and learn to write abstracts from a few words.'''
    _data = dict()
    _abstracts = []
    
    def __init__(self, ngram, randomize=False, seed=None, maxWords=200):
        self.ngram = ngram
        self.randomize = randomize
        self.seed = seed
        self.maxWords = maxWords
        if seed is not None:
            random.seed(seed)

    
    def fit(self, train):
        data_list = []
        for item in train:
            ind = 0
            textlist = self.__cleanup_text(item)
            self._abstracts.append(' '.join(textlist))
            while ind + self.ngram - 1 < len(textlist):
                currentTuple = tuple(textlist[ind:(ind+self.ngram-1)])
                nextWord = textlist[ind+self.ngram-1]
                data_list.append((tuple(textlist[ind:(ind+self.ngram-1)]),
                                        textlist[ind+self.ngram-1]))
                ind += 1
        data_list.sort()
        
        # Now that the list is sorted, convert it into a dictionary.
        # Since the list is sorted, each subsuquent entry is either new or a continuation of the previous one.

        previousKey = None
        for val in data_list:
            currentKey = val[0]
            if currentKey == previousKey:
                self._data[currentKey].append(val[1])
            else:
                self._data[currentKey] = [val[1]]
            previousKey = val[0]

    def find_next_word(self, phrase):
        '''phrase is a tuple of (typ. 3) words. Returns the next word.'''
        # Since data is sorted, it should be possible to search quickly. I'm not doing this properly.
        if phrase in self._data.keys():
            solution = self._data[phrase]
            
            # If there are more than one result, we either return the most popular option or
            # a random answer weighted by popularity.
            if self.randomize:
                return random.choice(solution)
            else:
                count = Counter(solution)
                return count.most_common(1)[0][0]
        else:
            return None
        
    def write_abstract(self, beginning):
        abstract = list(beginning)
        nextWord = ''
        ind = len(beginning) - self.ngram + 1
        while (nextWord is not None) and (ind < self.maxWords):
            nextWord = self.find_next_word(tuple(abstract[ind:(ind+self.ngram)]))
            if nextWord is None:
                break
            else:
                abstract.append(nextWord)
                ind += 1
        return ' '.join(abstract)
    
    def find_similar(self, abstract):
        '''Have I accidentally plagarized one of the articles?
        With too few choices, we will likely just copy an existing article.
        Returns the longest chunk highlighted in an existing article.'''
        first = 0
        last = 1
        abort = False
        longestChunk = ''
        longestChunkLength = 0

        totalLength = len(abstract.split())
        while first + longestChunkLength < totalLength:
            chunk = ' '.join(abstract.split()[first:last])
            if any(x for x in self._abstracts if chunk in x):
                if last - first + 1 > longestChunkLength:
                    longestChunk = chunk
                    longestChunkLength = last - first + 1
                last += 1
            else:
                first += 1
                last = first + 1
        
        best_abstract = (x for x in self._abstracts if longestChunk in x).next()
        return re.sub('(' + longestChunk + ')', '[[[' + longestChunk + ']]]', best_abstract)


    def __cleanup_text(self, text):
        # re from kaggle https://www.kaggle.com/c/word2vec-nlp-tutorial/details/part-1-for-beginners-bag-of-wordsi
        # Remove punctuation. Keep periods.
        # 
        # New trick: replace numbers with ##, which we will later replace with different numbers.
        pattern1 = re.compile("[0-9]")
        pattern2 = re.compile("[^a-zA-Z#.]")
        # The previous pattern could make multiple spaces. Turn multiple spaces into a single space.
        pattern3 = re.compile("(\s{2,})")
        return pattern3.sub(" ",
                            pattern2.sub(" ",
                                         pattern1.sub("#",
                                                      text.lstrip().rstrip()))).split(' ')

Let's make a bunch of abstracts that start in a way similar to existing abstracts, and find if they accidentally plagarize anything.

In [8]:
def WriteAbstract(ngram):
    writer = AbstractWriter(ngram=ngram, randomize=True, seed=42)
    writer.fit(abstract_train)
    for i in range(0,5):
        phrase = random.choice(writer._abstracts).split()[0:(ngram-1)]
        currentAbstract = writer.write_abstract(tuple(phrase))
        print 'New abstract: ' + currentAbstract
        print ''
        print 'Existing abstract: ' + writer.find_similar(currentAbstract)
        print ''
    return writer

In [9]:
WriteAbstract(4)

New abstract: We observed an enhanced atom dimer relaxation rate namely two resonances and two local minima. We use the Schwinger Keldysh formalism. We show that it displays at the critical number of particle in the disordered potential. In this system the role of the Ginzburg Landau equation SCGLE . We directly extract the static and dynamic structure factors through the renowned Tan relations. However a conceptual question remains open as to whether or not they are actually formed in supernova cores is still unclear. Here we solve the N body dynamics can be cast into the form of an exact flow equation for scale dependent composite operators. This equation allows for example a thermal quench starting from thermal states. We show that depending on the shaking parameter. It is shown that the repulsive interaction U from the metallic state through the superfluid Mott insulator transition for two dimensional Gross Pitaevskii equation. In the theoretical framework the regularization proces

<__main__.AbstractWriter instance at 0x108692680>

In [10]:
WriteAbstract(5)

New abstract: Existing techniques for synthesizing gauge fields are able to bring a two dimensional cloud of harmonically trapped bosonic atoms into a regime where the single atom Rabi frequency of the laser used for excitation to the Rydberg state within the volume of the higher energy pockets at the expense of another complementary property. Squeezed states were first demonstrated in optical fields and later with ensembles of pseudo spin # # Bose Einstein condensate by transferring a known quantity of energy to the condensate and generate quantized vortices under realistic experimental conditions and widely applicable for many systems. Finally this scheme can be used for the stabilization of particle number projection that is essential for reaching convergence in the size of the one particle reduced density matrix. We find that fluctuations induce a second order phase transition to a non magnetized SU # Fermi liquid phase.

Existing abstract: [[[Existing techniques for synthesizing g

<__main__.AbstractWriter instance at 0x10980a5f0>

In [11]:
WriteAbstract(6)

New abstract: Recently three dimensional topological quantum materials with gapless energy spectra have attracted considerable interests in many branches of physics. However no unambiguous experimental evidences for the existence of FFLO phases have been observed yet. Recently spin imbalanced ultracold degenerate Fermi gases have emerged as a new powerful platform for the observation of FFLO phases due to their high experimental controllability and the lack of disorder. However in three dimensional degenerate Fermi gases in the presence of spin orbit coupling. For the moving bright solitons their shapes are found to change with velocity due to the lack of any possible relaxation process in the Tonks Girardeau regime. We show that in the low density limit the ground state of this system by the Fisher information coherence visibility and entanglement entropy. These quantities are studied as functions of the interaction between bosons in different wells. The emergence of Schroedinger cat 

<__main__.AbstractWriter instance at 0x1098436c8>